In [216]:
import pandas as pd
import numpy as np

In [217]:
zillow_scrape=pd.read_csv('ZIllow Scrape SD 2019-04-14_184131.csv')
zillow_scrape.head()
zillow_scrape.dtypes

address            object
city               object
state              object
zip                 int64
price             float64
sqft              float64
bedrooms          float64
bathrooms         float64
days_on_zillow    float64
sale_type          object
url                object
dtype: object

In [218]:
shark_scrape=pd.read_csv('property_shark_excel_clean.csv')

shark_scrape['SQFT'] = shark_scrape['SQFT'].replace(',','',regex=True).astype(float)

shark_scrape=shark_scrape.drop(['Acorage'],axis=1)

for index, row in shark_scrape.iterrows():
    shark_scrape.loc[index,"Street Address"]= shark_scrape.loc[index,"address"].split(',')[0]
    shark_scrape.loc[index,"City"]= shark_scrape.loc[index,"address"].split(',')[1].strip('\n')
    shark_scrape.loc[index,"City"]= shark_scrape.loc[index,"address"].split(',')[1].strip('\n')

shark_scrape=shark_scrape.drop(['address'],axis=1)    
    
shark_scrape.head()

,neighborhood,SQFT,year built,Parcel ID,Property Class,Legal Description,units,bath,bed,zip,Street Address,City
0,Rancho Del Rey,2143.0,1979.0,640-141-22-00,Single Family Residence (11),LOT 19,1.0,2.5,4.0,91910,922 Calma Dr,Chula Vista
1,Rancho Del Rey,3551.0,2000.0,593-372-29-00,Single Family Residence (11),LOT 73,1.0,3.0,5.0,91910,933 Palencia Pl,Chula Vista
2,Rancho Del Rey,3466.0,1997.0,593-400-01-00,Single Family Residence (11),LOT 2,1.0,3.0,5.0,91910,707 Terra Nova Dr,Chula Vista
3,Rancho Del Rey,2671.0,2001.0,640-380-07-00,Single Family Residence (11),LOT 7,1.0,3.0,4.0,91910,993 Via Sinuoso,Chula Vista
4,Terra Nova,2762.0,1986.0,592-183-18-00,Single Family Residence (11),LOT 300,1.0,3.0,4.0,91910,522 Beacon Pl,Chula Vista


In [219]:
houses_merge=zillow_scrape.merge(shark_scrape, left_on='address', right_on='Street Address',suffixes=('_zillow', '_prop_shark'))

houses_merge.columns

# Find out how to convert dataframe to JSON, then push JSON to mongo


Index(['address', 'city', 'state', 'zip_zillow', 'price', 'sqft', 'bedrooms',
       'bathrooms', 'days_on_zillow', 'sale_type', 'url', 'neighborhood',
       'SQFT', 'year built', 'Parcel ID', 'Property Class',
       'Legal Description', 'units', 'bath', 'bed', 'zip_prop_shark',
       'Street Address', 'City'],
      dtype='object')

In [220]:
# Left off here

import json
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
db = client['SD_housing']
collection_houses = db['house_data']

collection_houses.drop()

collection_houses.insert_many(houses_merge.to_dict("records"))

In [ ]:
# # Use this & some loops to push to mongo

# # Import our pymongo library, which lets us connect our Flask app to our Mongo database.
# import pymongo

# # Create connection variable
# conn = 'mongodb://localhost:27017'

# # Pass connection to the pymongo instance.
# client = pymongo.MongoClient(conn)

# # Connect to a database. Will create one if not already available.
# db = client.SD_housing

# # Drops collection if available to remove duplicates
# db.zip_data.drop()

# # Creates a collection in the database and inserts two documents
# db.team.insert_many(
#     [
#         {
#             'player': 'Jessica',
#             'position': 'Point Guard'
#         },
#         {
#             'player': 'Mark',
#             'position': 'Center'
#         }
#     ]
# )

In [155]:
# Drop DB if necessary
